<a href="https://colab.research.google.com/github/JairsonAlbertoSami/desafio-junior-data-scientis/blob/main/Analise_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio Técnico - Cientista de Dados Júnior - Rio de Janeiro

Este código foi desenvolvido como parte do desafio técnico para a posição de **Cientista de Dados Júnior** no **Rio de Janeiro**. O objetivo deste desafio é demonstrar habilidades em manipulação e análise de dados com python


In [1]:
# Instala BigQuery e basedosdados

!pip install google-cloud-bigquery
!pip install basedosdados

In [2]:
# importar as bibliotecas necessarias

import pandas as pd
import basedosdados as bd

In [3]:
# id do projeto no google cloud

project_id = "datariojairson"

In [4]:
# Autenticar no google cloude

from google.colab import auth
auth.authenticate_user()

In [5]:
# Consulta dados 'bairro' e armazena no DataFrame.

df_dados_mestres = bd.read_sql("SELECT* FROM `datario.dados_mestres.bairro`",
                 billing_project_id = project_id)

Downloading: 100%|██████████| 164/164 [00:01<00:00, 116.98rows/s]


In [6]:
# Consulta chamados do dia '2023-04-01' e armazena no DataFrame.
data_especifica = '2023-04-01'

consulta_sql = f"""
SELECT *
FROM `datario.adm_central_atendimento_1746.chamado`
WHERE DATE(data_inicio) = '{data_especifica}'
"""

df_chamados = bd.read_sql(consulta_sql, billing_project_id=project_id)

Downloading: 100%|██████████| 1756/1756 [00:00<00:00, 2464.45rows/s]


# **Localização de chamados do 1746**

Utilize a tabela de Chamados do 1746 e a tabela de Bairros do Rio de Janeiro para as perguntas de 1-5.


